In [1]:
import numpy as np
import pandas as pd
from random import randint
from collections import Counter

In [2]:
def delete_duplicates(dataset, what_duplicates):
    c = Counter({el: count for el, count in Counter(what_duplicates).items() if count > 1})
    list_names = list()
    for ind in range(0,len(c.items())):
        list_names.append(c.items()[ind][0])
    list_ind = list()
    for i in range(0,len(list_names)):
        list_ind.append(dataset[what_duplicates == list_names[i]].index.values)
    indlist = list()
    for i in range(1,len(list_ind)):
        indlist.append(list_ind[i][0])
    dataset = dataset.drop(indlist,0)
    print 'del objects ' + str(len(c))
    dataset.index = range(0,len(dataset))
    return dataset

# Построение датасета с положительными примерами на основе парсинга файла логов

In [3]:
data = list()
with open('logs.log', 'r') as logfile: #файл логов надо скачать
    for line in logfile.readlines():
        if '+' in line:
            if 'http' in line:  #некоторые логи без картинок 
                data.append(line)
data = data[23:] #первые 23 кривые

In [4]:
users = list()
for line in data:
    users.append(line[44:].split(":")[0])
top = list()
for line in data:
    top.append(line[44:].split("/")[6].split(" ")[0] + '/' + line[44:].split("/")[7].split(" ")[0])
middle = list()
for line in data:
    middle.append(line[44:].split("/")[13].split(" ")[0] + '/' + line[44:].split("/")[14].split(" ")[0])
bottom = list()
for line in data:
    bottom.append(line[44:].split("/")[-2].split("\n")[0] + '/' + line[44:].split("/")[-1].split("\n")[0])

In [5]:
dataset = pd.DataFrame(data = [users,top,middle,bottom]).transpose()
columns = ['users','top','middle','bottom']
dataset.columns = columns
dataset.head(5)

,users,top,middle,bottom
0,alex_test,jampers/OO001EWPJQ27_4190535_1_v2.jpg,pants/IC647EWMWF91_4177819_1_v1.jpg,shoes/PT001AWNYQ43_4094966_1_v1.jpg
1,strak,jampers/TO029EWMOQ83_3943245_1_v1.jpg,pants/IN002EWLKW42_1_v1.jpg,shoes/CA086AWLTL36_1_v1.jpg
2,strak,jampers/BA007EWLBR09_1_v2.jpg,jeans/AL006EWIIB35_1_v1.jpg,high_heels/AN003AWKYU95_1_v1.jpg
3,strak,t-shirts/OO001EWMNY57_1_v2.jpg,shorts/OO001EWIFC88_1_v2.jpg,sneakers/AN048AWIXE16_1_v1.jpg
4,default_user,jampers/PE299EWNFQ41_3956785_1_v2.jpg,jeans/DO005EWKNA36_1_v1.jpg,boots/CA072AWNEN67_3972704_1_v1.jpg


In [6]:
#удалим одинаковые луки (которые дублировались в процессе ошибочных действий на сайте)
dataset = delete_duplicates(dataset,dataset.top + dataset.middle + dataset.bottom)

del objects 73


In [7]:
dataset.head(2)

,users,top,middle,bottom
0,alex_test,jampers/OO001EWPJQ27_4190535_1_v2.jpg,pants/IC647EWMWF91_4177819_1_v1.jpg,shoes/PT001AWNYQ43_4094966_1_v1.jpg
1,strak,jampers/TO029EWMOQ83_3943245_1_v1.jpg,pants/IN002EWLKW42_1_v1.jpg,shoes/CA086AWLTL36_1_v1.jpg


In [8]:
dataset.to_csv("csv/good_looks.csv",index=False)

# Построение отрицательных примеров случайным образом из всех соответствующих по типу(верх-низ) категориям

In [9]:
lamoda = pd.read_csv('csv/lamoda_without_spaces.csv')

In [10]:
lamoda.category.unique()

array(['Flats', 'Sandals', 'Ankle_boots', 'Shoes', 'Jackboots',
       'Snowjoggers', 'Sneakers', 'Loafers', 'New_year_shoes', 'Clogs',
       'Boots', 'Slip-ons', 'High_Heels', 'Uggs', 'Slippers',
       'Espadrilles', 'shirts', 'jampers', 'sweatshirts', 'tops',
       't-shirts', 'tunics', 'jeans', 'skirts', 'pants', 'shorts'], dtype=object)

In [11]:
lamoda.head(2)

,Unnamed: 0,file_name,category,brand,type
0,0,AL028AWKIN88_1_v1.jpg,Flats,Crosby,bottom
1,1,AL030AWHZR15_1_v1.jpg,Flats,Guapissima,bottom


In [12]:
lamoda = lamoda.drop(lamoda[lamoda.category == 'New_year_shoes'].index,0)
lamoda = lamoda.drop('Unnamed: 0',1)

In [13]:
#файл ламода содержит не уникальные file_name

In [14]:
lamoda.file_name.is_unique

False

In [15]:
#картинки с повтором одинаковые (просмотрено глазами), можно их удалять

In [16]:
lamoda = delete_duplicates(lamoda, lamoda.file_name)

del objects 35


In [17]:
top_min_index = lamoda[lamoda.type == 'top'].index[0]
top_max_index = lamoda[lamoda.type == 'top'].index[len(lamoda[lamoda.type == 'top'].index)-1]
middle_min_index = lamoda[lamoda.type == 'middle'].index[0]
middle_max_index = lamoda[lamoda.type == 'middle'].index[len(lamoda[lamoda.type == 'middle'].index)-1]
bottom_min_index = lamoda[lamoda.type == 'bottom'].index[0]
bottom_max_index = lamoda[lamoda.type == 'bottom'].index[len(lamoda[lamoda.type == 'bottom'].index)-1]

In [18]:
%%time
data_negative = list()
for i in range(0,len(dataset) * 5):
    top = randint(top_min_index,top_max_index)
    middle = randint(middle_min_index,middle_max_index)
    bottom = randint(bottom_min_index,bottom_max_index)
    data_negative.append([lamoda[lamoda.type == 'top'].loc[top].category + '/' +
                    lamoda[lamoda.type == 'top'].loc[top].file_name, 
                    lamoda[lamoda.type == 'middle'].loc[middle].category + '/' +
                    lamoda[lamoda.type == 'middle'].loc[middle].file_name, 
                    lamoda[lamoda.type == 'bottom'].loc[bottom].category + '/' +
                    lamoda[lamoda.type == 'bottom'].loc[bottom].file_name, 0])
#negative_dataset = pd.DataFrame(data = data_negative, columns = ['top','middle','bottom, y'])

CPU times: user 5min 14s, sys: 34.6 s, total: 5min 49s
Wall time: 7min 16s


In [19]:
negative_dataset = pd.DataFrame(data = data_negative, columns = ['top','middle','bottom', 'y'])

In [20]:
negative_dataset.head(3)

,top,middle,bottom,y
0,t-shirts/OO001EWHCY64_1_v2.jpg,pants/NI464EWJGB68_1_v1.jpg,High_Heels/DA016AWKKG26_1_v1.jpg,0
1,t-shirts/PE299EWNFQ75_3954957_1_v2.jpg,skirts/ZA004EWMRE08_4024690_1_v1.jpg,Sandals/DO030AWIVV36_1_v1.jpg,0
2,jampers/OO001EWOCV49_4071153_1_v3.jpg,pants/OO001EWLAT03_1_v1.jpg,Espadrilles/KA028AWIFX60_1_v1.jpg,0


# Соединяем датасет с положительными примерами и негативными

In [21]:
#Полный датасет с положительными и негативными примерами
y = pd.DataFrame(data = np.ones(len(dataset)), columns = ['y'])
Data = dataset.join(y)

In [22]:
Data = Data.append(negative_dataset)
Data.index = range(0,len(Data))

In [23]:
#удалим совпадения в нагенерированных сочетаниях и хороших луках
Data = delete_duplicates(Data,Data.top + Data.middle + Data.bottom)

del objects 4


In [25]:
Data.to_csv('csv/DATASET_3a.csv',index=False)

In [26]:
Data.head()

,bottom,middle,top,users,y
0,shoes/PT001AWNYQ43_4094966_1_v1.jpg,pants/IC647EWMWF91_4177819_1_v1.jpg,jampers/OO001EWPJQ27_4190535_1_v2.jpg,alex_test,1.0
1,shoes/CA086AWLTL36_1_v1.jpg,pants/IN002EWLKW42_1_v1.jpg,jampers/TO029EWMOQ83_3943245_1_v1.jpg,strak,1.0
2,high_heels/AN003AWKYU95_1_v1.jpg,jeans/AL006EWIIB35_1_v1.jpg,jampers/BA007EWLBR09_1_v2.jpg,strak,1.0
3,sneakers/AN048AWIXE16_1_v1.jpg,shorts/OO001EWIFC88_1_v2.jpg,t-shirts/OO001EWMNY57_1_v2.jpg,strak,1.0
4,boots/CA072AWNEN67_3972704_1_v1.jpg,jeans/DO005EWKNA36_1_v1.jpg,jampers/PE299EWNFQ41_3956785_1_v2.jpg,default_user,1.0


# Построение полного датасета с признаками global_pool

In [27]:
%%time
#приведем к более стандартному виду (file_name без названия папки)
for i in range(0,len(Data)):
    Data.top.loc[i] = Data.loc[i].top.split('/')[1]
    Data.middle.loc[i] = Data.loc[i].middle.split('/')[1]
    Data.bottom.loc[i] = Data.loc[i].bottom.split('/')[1]

//anaconda/lib/python2.7/site-packages/pandas/core/indexing.py:128: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


CPU times: user 16min 42s, sys: 6.69 s, total: 16min 49s
Wall time: 17min 59s


In [28]:
global_pool = pd.read_csv('csv/global_dataset.csv')

In [29]:
global_pool.head(2)

,file_name,path_to_image,category,brand,type,0,1,2,3,4,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,AL028AWKIN88_1_v1.jpg,/Users/Anna/Datasets/downloaded_images/lamoda/...,Flats,Crosby,bottom,1.340799,0.158480,0.232880,0.081647,0.084810,...,0.444015,0.231864,0.038863,0.574036,0.785982,1.726635,1.818732,0.0,0.93984,0.124891
1,AL030AWHZR15_1_v1.jpg,/Users/Anna/Datasets/downloaded_images/lamoda/...,Flats,Guapissima,bottom,1.972670,0.099346,0.018221,0.501671,0.450996,...,0.000000,1.166205,1.277552,5.164815,0.413175,1.173229,0.375062,0.0,2.22886,0.386066


In [30]:
global_pool = delete_duplicates(global_pool, global_pool.file_name)

del objects 35


In [31]:
#в некоторых файлах есть ошибки в названии

In [32]:
for i in range(0,len(Data)):
    if Data.loc[i].top[-1] != 'g':
        print Data.loc[i].top
        print i
        print 'top'
    if Data.loc[i].middle[-1] != 'g':
        print Data.loc[i].middle
        print i
        print 'middle'
    if Data.loc[i].bottom[-1] != 'g':
        print Data.loc[i].bottom
        print i
        print 'bottom'

VE024AWICQ56_1_v1.jpg

280
middle
FA024AWIRU73_1_v1.jp
866
bottom
RI369EWJGH75_1_v1.jpg + 
1634
bottom
UN012EWKTU22_1_v1.JPG
11107
top


In [33]:
Data.middle.loc[280] = Data.loc[280].middle[0:-1]

In [34]:
Data.bottom.loc[866] = Data.loc[866].bottom + 'g'

In [35]:
Data.bottom.loc[1634] = Data.loc[1634].bottom[0:-3]

In [36]:
%%time
y = list()
DATASET = pd.DataFrame(columns=[range(0, 3072)])
for i in range(0,len(Data)):
    a = global_pool[global_pool.file_name == Data.top[i]].drop(['file_name','path_to_image',
                                                               'category', 'brand','type'],1)
    b = global_pool[global_pool.file_name == Data.middle[i]].drop(['file_name','path_to_image',
                                                               'category', 'brand','type'],1)
    c = global_pool[global_pool.file_name == Data.bottom[i]].drop(['file_name','path_to_image',
                                                               'category', 'brand','type'],1)
    a.columns = [range(0,a.shape[1])]
    b.columns = [range(a.shape[1],b.shape[1]*2)]
    c.columns = [range(b.shape[1]*2, c.shape[1]*3)]
    if((len(c) == len(a)) == len(b)):
        c.index = [i]
        a.index = [i]
        b.index = [i]
        DATASET = DATASET.append(a.join(b.join(c)))
        y.append(Data.y[i])

CPU times: user 9min 56s, sys: 8min 59s, total: 18min 55s
Wall time: 18min 57s


In [37]:
DATASET = pd.concat([Data, DATASET], axis=1, join = 'inner',join_axes=[DATASET.index])

In [38]:
DATASET.index = range(0,len(DATASET))

In [39]:
DATASET.head(3)

,bottom,middle,top,users,y,0,1,2,3,4,...,3062,3063,3064,3065,3066,3067,3068,3069,3070,3071
0,PT001AWNYQ43_4094966_1_v1.jpg,IC647EWMWF91_4177819_1_v1.jpg,OO001EWPJQ27_4190535_1_v2.jpg,alex_test,1.0,1.035404,0.269563,0.143173,0.170981,0.308065,...,2.369041,1.575275,2.060973,1.533488,0.000842,0.168519,1.823179,0.331064,0.059062,0.000000
1,CA086AWLTL36_1_v1.jpg,IN002EWLKW42_1_v1.jpg,TO029EWMOQ83_3943245_1_v1.jpg,strak,1.0,1.492683,0.064760,0.071014,0.724826,2.543821,...,0.284060,0.086591,0.774355,1.586183,1.768617,0.440139,0.796940,0.066419,0.019898,0.431429
2,AN003AWKYU95_1_v1.jpg,AL006EWIIB35_1_v1.jpg,BA007EWLBR09_1_v2.jpg,strak,1.0,0.877401,0.075175,1.384997,0.238448,0.599227,...,0.023626,0.081537,0.131171,0.260097,0.030089,0.623181,0.450706,0.000000,0.030349,0.425907


In [40]:
DATASET.to_csv('csv/DATASET_3a_global.csv', index=False)